In [2]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from scipy.ndimage import grey_dilation

2024-10-17 10:25:00.253401: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-17 10:25:00.261723: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-17 10:25:00.327507: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-17 10:25:00.376071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 10:25:00.426371: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [3]:
#vamos aprende ler uma colecao de figuras

#Listar arquivos .png na pasta atual
current_directory = os.getcwd()+"/figuras/"

event_files=[[f for f in os.listdir(current_directory) if (f.endswith('.png') and ("event" in f) and ("nue" in f) and ("plane_"+str(i) in f))]
              for i in range(3)]
for i in range(3):
    event_files[i]=sorted(event_files[i])

hit_files=[[f for f in os.listdir(current_directory) if (f.endswith('.png') and ("hit" in f) and ("nue" in f) and ("plane_"+str(i) in f))]
              for i in range(3)]
for i in range(3):
    hit_files[i]=sorted(hit_files[i])

In [4]:
#selecionando figuras aleatorias

n_figuras=1000
n_total=len(event_files[0])

random=np.random.choice(n_total,n_figuras,replace=False)

x=[[] for _ in range(n_figuras)]
y=[[] for _ in range(n_figuras)]

x_end=272 # <--271
y_end=608 # <--610 

structure = np.ones((3, 3), dtype=np.uint8)  # Elemento estruturante 3x3

for j in range(n_figuras):
    for index in range(3):
        x[j].append(np.divide(grey_dilation(np.array(Image.open(current_directory+event_files[index][random[j]]).convert('L'), dtype=np.uint8),footprint=structure),215))
        y[j].append(np.divide(np.array(Image.open(current_directory+hit_files[index][random[j]]).convert('L')),215))

In [5]:
#deixando a saida binaria
for j in range(n_figuras):
    for i in range(3):
        y[j][i][(y[j][i] > 0.7)] = 1
        y[j][i][~((y[j][i] > 0.7) )] = 0 

In [6]:
#arrumando as dimennsoes da terceira imagem

from scipy.ndimage import zoom
new_matrix = np.zeros((271, 610))
for j in range(n_figuras):
    for index in range(3):
        x[j][index]=zoom(x[j][index],(x_end/len(x[j][index]),y_end/len(x[j][index][0])))
        y[j][index]=zoom(y[j][index],(x_end/len(y[j][index]),y_end/len(y[j][index][0])))

#deixando a saida binaria
for j in range(n_figuras):
    for index in range(3):
        y[j][index][(y[j][index] > 0.5)] = 1
        y[j][index][~((y[j][index] > 0.5) )] = 0 


In [7]:
#separando em treino e teste dados

x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=42)
y=[]
x=[]
#x_train,x_test,y_train,y_test = train_test_split(x,x_teste,random_state=42)
x_train=np.array(x_train)
x_test=np.array(x_test)
y_train=np.array(y_train)
y_test=np.array(y_test)


In [8]:
x_train = x_train.reshape((-1, 3, x_end,y_end))
x_test = x_test.reshape((-1, 3, x_end,y_end))
y_train = y_train.reshape((-1, 3, x_end,y_end))
y_test = y_test.reshape((-1, 3, x_end,y_end))

In [9]:
encoder = models.Sequential([
    layers.Conv2D(64, (5, 5), kernel_initializer="he_normal", padding='same', data_format='channels_first', input_shape=(3, x_end, y_end)),
    layers.BatchNormalization(axis=1),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.MaxPooling2D((2, 2), padding='same', data_format='channels_first'),
    layers.Conv2D(128, (5, 5), kernel_initializer="he_normal", padding='same', data_format='channels_first'),
    layers.BatchNormalization(axis=1),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.MaxPooling2D((2, 2), padding='same', data_format='channels_first'),
    layers.Conv2D(256, (3, 3), kernel_initializer="he_normal", padding='same', data_format='channels_first'),
    layers.BatchNormalization(axis=1),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.MaxPooling2D((2, 2), padding='same', data_format='channels_first'),
    layers.Conv2D(300, (3, 3), kernel_initializer="he_normal", padding='same', data_format='channels_first'),
    layers.BatchNormalization(axis=1),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.MaxPooling2D((2, 2), padding='same', data_format='channels_first')
])

decoder = models.Sequential([
    layers.Conv2D(300, (3, 3), kernel_initializer="he_normal", padding='same', data_format='channels_first'),
    layers.BatchNormalization(axis=1),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.UpSampling2D((2, 2), data_format='channels_first'),
    layers.Conv2D(256, (3, 3), kernel_initializer="he_normal", padding='same', data_format='channels_first'),
    layers.BatchNormalization(axis=1),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.UpSampling2D((2, 2), data_format='channels_first'),
    layers.Conv2D(128, (3, 3), kernel_initializer="he_normal", padding='same', data_format='channels_first'),
    layers.BatchNormalization(axis=1),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.UpSampling2D((2, 2), data_format='channels_first'),
    layers.Conv2D(64, (3, 3), kernel_initializer="he_normal", padding='same', data_format='channels_first'),
    layers.BatchNormalization(axis=1),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.UpSampling2D((2, 2), data_format='channels_first'),
    layers.Conv2D(3, (3, 3), activation='sigmoid', kernel_initializer="he_normal", padding='same', data_format='channels_first')
])

autoencoder2 = models.Sequential([encoder, decoder])

autoencoder2.compile(
    loss='mean_squared_error',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
)

/home/gabriel/.local/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
history = autoencoder2.fit(x_train,x_train, epochs=10, batch_size=10, validation_data=[x_test,x_test])

Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1159s 15s/step - loss: 0.0275 - val_loss: 0.0158
Epoch 2/10
24/75 ━━━━━━━━━━━━━━━━━━━━ 12:24 15s/step - loss: 0.0036

In [ ]:
#printa imagem original, imagem recuperada e imagem esperada

index=15


prediction=autoencoder2.predict(np.expand_dims(x_test[index]))
plt.figure(figsize=(12, 6))

for j in range(3):
    # Exibir a imagem original
    plt.subplot(3, 3, j*3+0)
    plt.imshow(x_test[index][j], cmap='gray')  # Adapte o cmap se necessário
    plt.title('Original Image')
    plt.axis('off')  # Desliga os eixos
    plt.tight_layout()

    #Exibir Imagem esperada
    plt.subplot(2, 2, j*3+1)
    plt.imshow(y_test[index][j], cmap='gray')  # Adapte o cmap se necessário
    plt.title('Expected image')
    plt.axis('off')  # Desliga os eixos
    # Mostrar as imagens
    plt.tight_layout()

    plt.subplot(2, 2, j*3+2)
    plt.imshow(prediction[0][j], cmap='gray')  # Adapte o cmap se necessário
    plt.title('Model 2 reconstruction')
    plt.axis('off')  # Desliga os eixos
    # Mostrar as imagens
    plt.tight_layout()

plt.show()
